# 一、视频分帧
## 提取视频帧
将视频分割为连续的图像帧，便于后续处理。

In [2]:
import cv2
import os

# 打开视频文件
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

# 创建保存帧的文件夹
frames_folder = "frames"
os.makedirs(frames_folder, exist_ok=True)

# 逐帧提取
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # 保存帧到frames文件夹
    frame_path = os.path.join(frames_folder, f"frame_{frame_count:04d}.png")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

cap.release()
print(f"提取完成，共 {frame_count} 帧")

提取完成，共 714 帧


# 二、动态目标标注（二值掩膜）
## 标注内容
  标注内容：为每帧生成动态目标的二值掩膜（0为背景，1为动态目标）

In [4]:
# LabelMe：手动标注动态目标轮廓，生成掩膜。
import os
import cv2
import numpy as np
import json
from labelme import utils

def json_to_mask(json_path, output_dir):
    # 加载标注文件
    with open(json_path, "r") as f:
        data = json.load(f)
    
    # 解析图像尺寸
    img_shape = (data["imageHeight"], data["imageWidth"])
    
    # 生成掩膜
    mask = np.zeros(img_shape, dtype=np.uint8)
    for shape in data["shapes"]:
        if shape["label"] == "dynamic_target":  # 仅处理动态目标
            points = shape["points"]
            points = np.array(points, dtype=np.int32)
            cv2.fillPoly(mask, [points], 1)
    
    # 保存掩膜
    mask_path = os.path.join(output_dir, os.path.basename(json_path).replace(".json", "_mask.png"))
    cv2.imwrite(mask_path, mask * 255)  # 将掩膜保存为二值图像

# 批量处理标注文件
json_dir = "annotations"
output_dir = "masks"
os.makedirs(output_dir, exist_ok=True)
for json_file in os.listdir(json_dir):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_dir, json_file)
        json_to_mask(json_path, output_dir)